## Documentación uso de Cartera

### Los import necesarios

In [1]:
import pyodbc

from Activo import *
from Accion import *
from Bono import *
from Derivado import *
from DerivadosTipos.DerivadosSCC import *

from Cartera import *

### Definimos la conexión a base de datos

In [2]:
server = '172.16.1.38'
username = 'sa'
password = 'qwerty123'
driver = '{ODBC Driver 17 for SQL Server}'
cn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';UID=' + username + ';PWD=' + password)

### Primero, definimos los inputs (activos) que se le entregarán a la cartera

#### Construimos una acción
 A partir de un archivo xlsx se extrae la columna 'Adj Close' para definir el historico de la acción. Hay que insertar la ruta correcta del archivo.

In [3]:
#archivo = pd.read_excel('C:\\Users\\groso\\Desktop\\Practica\\Intento\\Cartera_definitiva\\AMZN.xlsx')    
archivo = pd.read_excel('C:\\Users\\Lenovo\\Documents\\Universidad\\Practica\\Cartera_V2\\Practica\\Intento\\Cartera\\AMZN.xlsx')
columnas = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
archivo = archivo[columnas][:60]
accion = pd.DataFrame()
accion['Moneda'] = ['USD']
accion['Historico'] = [[archivo["Adj Close"]]]
accion['Nombre'] = "Amazon"
accion['Inversion'] = [500000]

#### Definimos un bono

In [4]:
bono = pd.DataFrame()
bono['Riesgo'] = ['AAA']  
bono['Moneda'] = ["CLP"]
bono['TablaDesarrollo'] = ["1#25-09-2018#2,2252#0#100#2,2252|2#25-03-2019#2,2252#0#100#2,2252|3#25-09-2019#2,2252#0#100#2,2252|4#25-03-2020#2,2252#0#100#2,2252|5#25-09-2020#2,2252#0#100#2,2252|6#25-03-2021#2,2252#100#0#102,2252"]
bono['Convencion'] = ["ACT360"]
bono['FechaEmision'] = ['2018-02-20']

#### Declaramos un derivado

In [26]:
info_derivado = dict()
info_derivado["Tipo"] = 'SCC'
info_derivado["ID_Key"] = ""
info_derivado["Administradora"] = "Admin"
info_derivado["Fondo"] = "Fondo"
info_derivado["Contraparte"] = "Contraparte"
info_derivado["ID"] = 3
info_derivado["Nemotecnico"] = ""
info_derivado["Mercado"] = "Local"
fecha = datetime.date(2019, 12, 20)
hora = '1700'
info_derivado["FechaEfectiva"] = '12/11/2019'
info_derivado["FechaVenc"] = '12/11/2021'
info_derivado["AjusteFeriados"] = "CL"
info_derivado["NocionalActivo"] = 10*(10**9)
info_derivado["MonedaActivo"] = 'CLP'
info_derivado["MonedaBase"] = 'CLP'
info_derivado["TipoTasaActivo"] = 'Fija'
info_derivado["TipoTasaPasivo"] = 'Flotante'
info_derivado["TasaActivo"] = 1.45
info_derivado["TasaPasivo"] = -1000
info_derivado["FrecuenciaActivo"] = "Semi annual"
info_derivado["FrecuenciaPasivo"] = info_derivado["FrecuenciaActivo"]

info1 = pd.DataFrame([info_derivado])
derivado_info = DerivadosSCC(fecha, hora, info1, cn)
print(derivado_info.get_fecha_efectiva())
derivado = pd.DataFrame()
derivado['Derivado'] = [derivado_info]

0    12/11/2019
Name: FechaEfectiva, dtype: object


### La cartera

Ahora entregamos estos activos a la cartera para que esta los pueda procesar. A Cartera se le debe entregar un dataFrame con las acciones, otro para bonos y aparte para derivados, luego se declara la moneda con la que se desea evaluar los indicadores de la cartera, la fecha a valorizar y la conexión a base de datos.

In [6]:
cartera = Cartera(accion, bono, derivado, 'CLP', datetime.date(2019,2,1), cn)

Para inicializar los cálculos, se debe pedir que se seteen los historicos, retornos y volatilidades respectivas.

In [7]:
cartera.set_hist_ret_vol_totales()

#### Acciones

In [8]:
acciones = cartera.get_acciones()
acciones

In [9]:
accion = acciones[0]
accion.get_retornos()

,Amazon
0,0.000000
1,0.012803
2,-0.013271
3,-0.020641
4,0.032355
5,-0.022802
6,-0.016222
7,0.006927
8,-0.010181
9,0.001867


In [10]:
accion.get_volatilidad()

,0
Amazon,0.017507


#### Bonos


In [11]:
bonos = cartera.get_bonos()
bonos

In [12]:
bono = bonos[0]
bono.get_historicos()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,CLP#3600#AAA,CLP#5400#AAA,CLP#7200#AAA,CLP#10800#AAA
0,0.987399,0.957999,0.911080,0.822914,0.675853,0.556498,0.458324,0.377475,0.256047,0.173681,0.143044,0.054206,0.020541,0.002950
1,0.987549,0.957278,0.913511,0.831901,0.689907,0.572151,0.474494,0.393505,0.270639,0.186136,0.154366,0.060555,0.023755,0.003656
2,0.987764,0.959484,0.912896,0.821228,0.669218,0.550894,0.455016,0.376130,0.257114,0.175767,0.145326,0.056154,0.021698,0.003240
3,0.988847,0.959121,0.913728,0.829248,0.683021,0.562584,0.463384,0.381676,0.258943,0.175676,0.144700,0.054858,0.020798,0.002989
4,0.988987,0.958709,0.914102,0.831050,0.686913,0.567778,0.469305,0.387912,0.265026,0.181069,0.149665,0.057744,0.022279,0.003316
5,0.988214,0.958132,0.914597,0.833389,0.691972,0.574553,0.477059,0.396108,0.273085,0.188270,0.156323,0.061693,0.024347,0.003792
6,0.988326,0.958285,0.914893,0.833932,0.692876,0.575681,0.478309,0.397406,0.274339,0.189383,0.157350,0.062302,0.024668,0.003867
7,0.988307,0.958026,0.914258,0.832647,0.690639,0.572851,0.475152,0.394116,0.271148,0.186548,0.154732,0.060749,0.023850,0.003676
8,0.988267,0.958943,0.916592,0.837431,0.699036,0.583512,0.487081,0.406586,0.283305,0.197404,0.164781,0.066783,0.027066,0.004446
9,0.987780,0.959208,0.917918,0.840600,0.704955,0.591200,0.495801,0.415796,0.292433,0.205671,0.172483,0.071550,0.029681,0.005108


In [13]:
bono.get_volatilidad()

,0
CLP#30#AAA,0.000421
CLP#90#AAA,0.000938
CLP#180#AAA,0.001310
CLP#360#AAA,0.002483
CLP#720#AAA,0.006933
CLP#1080#AAA,0.011199
CLP#1440#AAA,0.014678
CLP#1800#AAA,0.017469
CLP#2520#AAA,0.021908
CLP#3240#AAA,0.025845


In [14]:
bono.get_correlacion()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,CLP#3600#AAA,CLP#5400#AAA,CLP#7200#AAA,CLP#10800#AAA
CLP#30#AAA,1.000000,0.664950,0.236762,0.055153,0.082949,0.109262,0.126165,0.139083,0.158859,0.172769,0.177985,0.193306,0.199846,0.204900
CLP#90#AAA,0.664950,1.000000,0.748364,0.030791,-0.240620,-0.266673,-0.271728,-0.273090,-0.271795,-0.267308,-0.264469,-0.249937,-0.238451,-0.223713
CLP#180#AAA,0.236762,0.748364,1.000000,0.519541,0.084677,-0.011696,-0.053609,-0.079528,-0.113971,-0.137166,-0.146057,-0.174267,-0.188387,-0.201692
CLP#360#AAA,0.055153,0.030791,0.519541,1.000000,0.853766,0.767780,0.722491,0.693397,0.650431,0.613373,0.596279,0.526976,0.480562,0.425858
CLP#720#AAA,0.082949,-0.240620,0.084677,0.853766,1.000000,0.986413,0.969675,0.954539,0.922714,0.887407,0.869635,0.792518,0.738163,0.672320
CLP#1080#AAA,0.109262,-0.266673,-0.011696,0.767780,0.986413,1.000000,0.996218,0.988513,0.965047,0.934390,0.918224,0.845631,0.793186,0.728844
CLP#1440#AAA,0.126165,-0.271728,-0.053609,0.722491,0.969675,0.996218,1.000000,0.997698,0.982247,0.957457,0.943629,0.878759,0.830380,0.770043
CLP#1800#AAA,0.139083,-0.273090,-0.079528,0.693397,0.954539,0.988513,0.997698,1.000000,0.992478,0.974075,0.962882,0.906991,0.863488,0.808073
CLP#2520#AAA,0.158859,-0.271795,-0.113971,0.650431,0.922714,0.965047,0.982247,0.992478,1.000000,0.994389,0.988578,0.951355,0.918189,0.873363
CLP#3240#AAA,0.172769,-0.267308,-0.137166,0.613373,0.887407,0.934390,0.957457,0.974075,0.994389,1.000000,0.998973,0.978593,0.954914,0.919943


In [15]:
bono.get_volatilidad_general()

array([[0.00637515]])

#### Derivados

In [16]:
derivados = cartera.get_derivados()
derivados

In [17]:
derivado = derivados[0]
derivado.get_volatilidad()

,0
CLP#30,0.000314
CLP#90,0.000430
CLP#180,0.000784
CLP#360,0.000980
CLP#720,0.001190
CLP#1080,0.001783
CLP#1440,0.002557
CLP#1800,0.003090
CLP#2520,0.004224
CLP#3240,0.005154


In [18]:
derivado.get_correlacion()

,CLP#30,CLP#90,CLP#180,CLP#360,CLP#720,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800
CLP#30,1.000000,0.673652,0.674033,0.626206,0.678940,0.661205,0.651159,0.626690,0.582490,0.532699,0.540618,0.548712,0.487762,0.483383
CLP#90,0.673652,1.000000,0.863527,0.871404,0.621831,0.649003,0.604989,0.613424,0.598986,0.601185,0.588761,0.524174,0.608049,0.610769
CLP#180,0.674033,0.863527,1.000000,0.946121,0.726632,0.735600,0.717510,0.721892,0.661689,0.647525,0.633013,0.557533,0.642666,0.645385
CLP#360,0.626206,0.871404,0.946121,1.000000,0.678863,0.698730,0.656433,0.653961,0.615467,0.598176,0.575808,0.499005,0.583229,0.586040
CLP#720,0.678940,0.621831,0.726632,0.678863,1.000000,0.990942,0.986336,0.974294,0.945289,0.920658,0.905554,0.902531,0.890554,0.888097
CLP#1080,0.661205,0.649003,0.735600,0.698730,0.990942,1.000000,0.990889,0.977933,0.956939,0.937525,0.917157,0.909916,0.912106,0.910318
CLP#1440,0.651159,0.604989,0.717510,0.656433,0.986336,0.990889,1.000000,0.992056,0.967716,0.946130,0.930310,0.922744,0.924566,0.922711
CLP#1800,0.626690,0.613424,0.721892,0.653961,0.974294,0.977933,0.992056,1.000000,0.973660,0.953503,0.941969,0.922307,0.934987,0.933685
CLP#2520,0.582490,0.598986,0.661689,0.615467,0.945289,0.956939,0.967716,0.973660,1.000000,0.984212,0.976939,0.960180,0.962021,0.960163
CLP#3240,0.532699,0.601185,0.647525,0.598176,0.920658,0.937525,0.946130,0.953503,0.984212,1.000000,0.993469,0.977183,0.977396,0.975434


In [19]:
derivado.get_volatilidad_general()

0.0012961716340885113

Por ejemplo, si deseamos saber la volatilidad de la cartera, se debe invocar al siguiente método:

In [20]:
cartera.set_volatilidad_cartera()
volatilidad = cartera.get_volatilidad_cartera()
volatilidad

0.0019214554238695754

In [21]:
cartera.get_historicos_totales()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
0,0.987399,0.957999,0.911080,0.822914,0.675853,0.556498,0.458324,0.377475,0.256047,0.173681,...,0.945264,0.920908,0.892927,0.830382,0.764963,0.736152,0.615813,0.521599,0.376624,133.899994
1,0.987549,0.957278,0.913511,0.831901,0.689907,0.572151,0.474494,0.393505,0.270639,0.186136,...,0.945271,0.920921,0.893143,0.831261,0.766409,0.738088,0.617735,0.525645,0.381113,134.690002
2,0.987764,0.959484,0.912896,0.821228,0.669218,0.550894,0.455016,0.376130,0.257114,0.175767,...,0.945134,0.921105,0.893373,0.831533,0.766046,0.737300,0.617233,0.524469,0.379802,132.250000
3,0.988847,0.959121,0.913728,0.829248,0.683021,0.562584,0.463384,0.381676,0.258943,0.175676,...,0.945224,0.921186,0.893525,0.831692,0.766129,0.737445,0.617343,0.523926,0.379195,130.000000
4,0.988987,0.958709,0.914102,0.831050,0.686913,0.567778,0.469305,0.387912,0.265026,0.181069,...,0.945505,0.921717,0.894395,0.832008,0.767146,0.738167,0.618230,0.525643,0.381076,133.520004
5,0.988214,0.958132,0.914597,0.833389,0.691972,0.574553,0.477059,0.396108,0.273085,0.188270,...,0.945408,0.921966,0.894696,0.833058,0.768410,0.739573,0.620019,0.527619,0.383226,130.309998
6,0.988326,0.958285,0.914893,0.833932,0.692876,0.575681,0.478309,0.397406,0.274339,0.189383,...,0.944585,0.920531,0.892919,0.830680,0.766395,0.737442,0.616786,0.525871,0.381424,127.349998
7,0.988307,0.958026,0.914258,0.832647,0.690639,0.572851,0.475152,0.394116,0.271148,0.186548,...,0.942910,0.917479,0.888932,0.825432,0.760885,0.731264,0.610021,0.516319,0.370984,129.110001
8,0.988267,0.958943,0.916592,0.837431,0.699036,0.583512,0.487081,0.406586,0.283305,0.197404,...,0.941894,0.915942,0.886688,0.822472,0.757257,0.726994,0.605628,0.510754,0.364994,127.349998
9,0.987780,0.959208,0.917918,0.840600,0.704955,0.591200,0.495801,0.415796,0.292433,0.205671,...,0.942167,0.916473,0.887556,0.823905,0.759022,0.728811,0.608453,0.512011,0.366222,127.139999


In [22]:
cartera.get_retornos_totales()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000151,-0.000753,0.002665,0.010862,2.058162e-02,0.027739,0.034671,0.041589,0.055424,0.069258,...,7.038878e-06,0.000015,0.000242,0.001058,0.001888,0.002627,0.003116,0.007727,0.011849,0.012803
2,0.000217,0.002302,-0.000674,-0.012913,-3.044799e-02,-0.037861,-0.041915,-0.045160,-0.051267,-0.057319,...,-1.444339e-04,0.000199,0.000258,0.000327,-0.000474,-0.001068,-0.000813,-0.002239,-0.003447,-0.013271
3,0.001096,-0.000379,0.000911,0.009718,2.041612e-02,0.020998,0.018222,0.014639,0.007088,-0.000517,...,9.527514e-05,0.000088,0.000169,0.000192,0.000109,0.000196,0.000179,-0.001035,-0.001599,-0.020641
4,0.000141,-0.000429,0.000409,0.002171,5.681811e-03,0.009190,0.012698,0.016205,0.023220,0.030234,...,2.964891e-04,0.000576,0.000974,0.000379,0.001326,0.000979,0.001436,0.003271,0.004949,0.032355
5,-0.000782,-0.000602,0.000542,0.002811,7.338297e-03,0.011863,0.016386,0.020909,0.029956,0.039002,...,-1.026542e-04,0.000270,0.000336,0.001262,0.001646,0.001903,0.002888,0.003751,0.005625,-0.022802
6,0.000113,0.000159,0.000323,0.000651,1.305844e-03,0.001961,0.002616,0.003272,0.004582,0.005892,...,-8.700799e-04,-0.001557,-0.001988,-0.002859,-0.002626,-0.002885,-0.005228,-0.003318,-0.004712,-0.016222
7,-0.000019,-0.000270,-0.000695,-0.001542,-3.234773e-03,-0.004928,-0.006621,-0.008313,-0.011699,-0.015085,...,-1.775740e-03,-0.003322,-0.004474,-0.006337,-0.007215,-0.008413,-0.011028,-0.018331,-0.027753,0.006927
8,-0.000041,0.000957,0.002550,0.005729,1.208504e-02,0.018440,0.024795,0.031150,0.043859,0.056568,...,-1.077137e-03,-0.001676,-0.002527,-0.003593,-0.004780,-0.005856,-0.007227,-0.010836,-0.016280,-0.010181
9,-0.000493,0.000277,0.001446,0.003776,8.432997e-03,0.013089,0.017745,0.022401,0.031713,0.041024,...,2.889848e-04,0.000579,0.000978,0.001741,0.002328,0.002496,0.004654,0.002458,0.003359,0.001867


In [23]:
cartera.get_volatilidades_totales()

,0
CLP#30#AAA,0.000421
CLP#90#AAA,0.000938
CLP#180#AAA,0.001310
CLP#360#AAA,0.002483
CLP#720#AAA,0.006933
CLP#1080#AAA,0.011199
CLP#1440#AAA,0.014678
CLP#1800#AAA,0.017469
CLP#2520#AAA,0.021908
CLP#3240#AAA,0.025845


In [24]:
cartera.get_correlacion()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
CLP#30#AAA,1.000000,0.664950,0.236762,0.055153,0.082949,0.109262,0.126165,0.139083,0.158859,0.172769,...,0.125064,0.130114,0.127596,0.121674,0.128837,0.117075,0.123856,0.140688,0.140939,0.075144
CLP#90#AAA,0.664950,1.000000,0.748364,0.030791,-0.240620,-0.266673,-0.271728,-0.273090,-0.271795,-0.267308,...,0.077609,0.081688,0.088958,0.061811,0.061086,0.029644,0.025847,0.102515,0.106019,-0.120195
CLP#180#AAA,0.236762,0.748364,1.000000,0.519541,0.084677,-0.011696,-0.053609,-0.079528,-0.113971,-0.137166,...,0.061026,0.074847,0.087841,0.051137,0.010744,-0.006486,-0.008460,0.078004,0.082192,-0.137679
CLP#360#AAA,0.055153,0.030791,0.519541,1.000000,0.853766,0.767780,0.722491,0.693397,0.650431,0.613373,...,0.109500,0.134854,0.142974,0.123621,0.067624,0.073285,0.068740,0.120968,0.123272,0.068199
CLP#720#AAA,0.082949,-0.240620,0.084677,0.853766,1.000000,0.986413,0.969675,0.954539,0.922714,0.887407,...,0.094275,0.108369,0.107611,0.109550,0.073241,0.080922,0.077290,0.100920,0.101817,0.123247
CLP#1080#AAA,0.109262,-0.266673,-0.011696,0.767780,0.986413,1.000000,0.996218,0.988513,0.965047,0.934390,...,0.087032,0.098546,0.095948,0.104094,0.073295,0.081559,0.079268,0.095124,0.095644,0.137718
CLP#1440#AAA,0.126165,-0.271728,-0.053609,0.722491,0.969675,0.996218,1.000000,0.997698,0.982247,0.957457,...,0.086791,0.098326,0.095702,0.107479,0.079112,0.088083,0.086585,0.099155,0.099498,0.152359
CLP#1800#AAA,0.139083,-0.273090,-0.079528,0.693397,0.954539,0.988513,0.997698,1.000000,0.992478,0.974075,...,0.089254,0.101598,0.099585,0.114241,0.087482,0.097164,0.096252,0.106998,0.107227,0.167250
CLP#2520#AAA,0.158859,-0.271795,-0.113971,0.650431,0.922714,0.965047,0.982247,0.992478,1.000000,0.994389,...,0.096051,0.110556,0.110332,0.129705,0.105472,0.116443,0.116457,0.125021,0.125089,0.194762
CLP#3240#AAA,0.172769,-0.267308,-0.137166,0.613373,0.887407,0.934390,0.957457,0.974075,0.994389,1.000000,...,0.101902,0.118366,0.119905,0.142886,0.120816,0.132778,0.133516,0.140616,0.140566,0.216464


In [25]:
cartera.get_covarianza()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1.775432e-07,2.627726e-07,1.306937e-07,5.771085e-08,2.423212e-07,5.155702e-07,7.802968e-07,1.023770e-06,1.466443e-06,1.881432e-06,...,9.398054e-08,1.402124e-07,1.661186e-07,2.165803e-07,2.798120e-07,2.813691e-07,4.722325e-07,6.826222e-07,0.000001,5.543273e-07
1,2.627726e-07,8.795846e-07,9.194787e-07,7.171238e-08,-1.564587e-06,-2.800822e-06,-3.740619e-06,-4.474263e-06,-5.584473e-06,-6.479201e-06,...,1.298086e-07,1.959330e-07,2.577833e-07,2.448889e-07,2.952944e-07,1.585727e-07,2.193511e-07,1.107129e-06,0.000002,-1.973534e-06
2,1.306937e-07,9.194787e-07,1.716249e-06,1.690239e-06,7.690988e-07,-1.715942e-07,-1.030857e-06,-1.820064e-06,-3.271052e-06,-4.644150e-06,...,1.425785e-07,2.507693e-07,3.555634e-07,2.830020e-07,7.254761e-08,-4.846338e-08,-1.002908e-07,1.176739e-06,0.000002,-3.157755e-06
3,5.771085e-08,7.171238e-08,1.690239e-06,6.167023e-06,1.469960e-05,2.135216e-05,2.633538e-05,3.008134e-05,3.538678e-05,3.936703e-05,...,4.849595e-07,8.564702e-07,1.097048e-06,1.296869e-06,8.655909e-07,1.038039e-06,1.544670e-06,3.459236e-06,0.000005,2.965090e-06
4,2.423212e-07,-1.564587e-06,7.690988e-07,1.469960e-05,4.806812e-05,7.658686e-05,9.867891e-05,1.156111e-04,1.401515e-04,1.590089e-04,...,1.165668e-06,1.921504e-06,2.305226e-06,3.208546e-06,2.617316e-06,3.200046e-06,4.848874e-06,8.057088e-06,0.000012,1.495981e-05
5,5.155702e-07,-2.800822e-06,-1.715942e-07,2.135216e-05,7.658686e-05,1.254106e-04,1.637537e-04,1.933869e-04,2.367650e-04,2.704364e-04,...,1.738197e-06,2.822385e-06,3.319971e-06,4.924486e-06,4.230756e-06,5.209553e-06,8.032553e-06,1.226670e-05,0.000018,2.700091e-05
6,7.802968e-07,-3.740619e-06,-1.030857e-06,2.633538e-05,9.867891e-05,1.637537e-04,2.154462e-04,2.558267e-04,3.158583e-04,3.632110e-04,...,2.271940e-06,3.691018e-06,4.340325e-06,6.664389e-06,5.985326e-06,7.374312e-06,1.150001e-05,1.675923e-05,0.000025,3.915236e-05
7,1.023770e-06,-4.474263e-06,-1.820064e-06,3.008134e-05,1.156111e-04,1.933869e-04,2.558267e-04,3.051791e-04,3.798396e-04,4.397847e-04,...,2.780720e-06,4.539111e-06,5.375268e-06,8.430735e-06,7.877167e-06,9.681493e-06,1.521502e-05,2.152400e-05,0.000032,5.115217e-05
8,1.466443e-06,-5.584473e-06,-3.271052e-06,3.538678e-05,1.401515e-04,2.367650e-04,3.158583e-04,3.798396e-04,4.799592e-04,5.630264e-04,...,3.752794e-06,6.194340e-06,7.468487e-06,1.200400e-05,1.191011e-05,1.455038e-05,2.308622e-05,3.153956e-05,0.000047,7.470080e-05
9,1.881432e-06,-6.479201e-06,-4.644150e-06,3.936703e-05,1.590089e-04,2.704364e-04,3.632110e-04,4.397847e-04,5.630264e-04,6.679443e-04,...,4.696825e-06,7.823573e-06,9.574932e-06,1.560011e-05,1.609419e-05,1.957298e-05,3.122410e-05,4.184820e-05,0.000063,9.794362e-05
